In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding, Input, Dot, Dense, Flatten, Multiply, Concatenate
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError
import datetime

# Preprocess data

In [2]:
def preprocess_data(data_path):
    header_data = ['user_id', 'item_id', 'rating', 'timestamp']
    data = pd.read_csv(data_path, sep='\t', names=header_data)
    data = data.drop("timestamp", axis=1).astype(int)
    
    num_items = data["item_id"].max()
    num_users = data["user_id"].max()
    
    y = data["rating"]
    y = np.where(y==5, 1, 0)

    X = data.drop("rating", axis=1)
    # Minus 1 so the index starts from 0
    X = X-1
    X = X.astype('category')
    
    return X, y, num_items, num_users

In [3]:
X_train, y_train, num_items, num_users = preprocess_data("data/ub.base")
X_test , y_test, _, _ = preprocess_data("data/ub.test")

In [4]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(90570, 2) (90570,) (9430, 2) (9430,)


# Negative Sampling

In [5]:
def get_train_instances(users, items, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = users.shape[0]
    for (u, i) in zip(users, items):
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
#             while train.has_key((u, j)):
#                 j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [6]:
user_input_training, item_input_training, labels_training \
    = get_train_instances(X_train["user_id"].to_numpy(), X_train["item_id"].to_numpy(), 4)
user_input_test, item_input_test, labels_test \
    = get_train_instances(X_test["user_id"].to_numpy(), X_test["item_id"].to_numpy(), 4)

In [7]:
print(len(user_input_training), len(user_input_test))

452850 47150


# Get model

In [8]:
# Dot product model
def get_model_dot_product(num_users, num_items, latent_dim, regs=[0,0]):
    
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    
    # Embedding layers
    user_embedding_layer = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding',
                                  embeddings_initializer='uniform', input_length=1)
    user_embedding = user_embedding_layer(user_input)
    
    item_embedding_layer = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding',
                                  embeddings_initializer='uniform', input_length=1)
    item_embedding = item_embedding_layer(item_input)
    
    # Flatten embeddings
    user_latent = Flatten()(user_embedding)
    item_latent = Flatten()(item_embedding)
    
    # Dot product layer
    dotted = Dot(axes=1)([user_latent, item_latent])
    
    # add the sigmoid output layer
    output = Dense(1, activation='sigmoid')(dotted)
    
    model = Model([user_input, item_input], output)
    
    return model

In [9]:
# GMF model
def get_model_gmf(num_users, num_items, latent_dim, regs=[0,0]):
    
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    
    # Embedding layers
    user_embedding_layer = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding',
                                  embeddings_initializer='uniform', input_length=1)
    user_embedding = user_embedding_layer(user_input)
    
    item_embedding_layer = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding',
                                  embeddings_initializer='uniform', input_length=1)
    item_embedding = item_embedding_layer(item_input)
    
    # Flatten embeddings
    user_latent = Flatten()(user_embedding)
    item_latent = Flatten()(item_embedding)
    
    # Element wise multiply layer
    multiply = Multiply()([user_latent, item_latent])
    
    # add the sigmoid output layer
    output = Dense(1, activation='sigmoid')(multiply)
    
    model = Model([user_input, item_input], output)
    
    return model

In [10]:
# MLP model
def get_model_mlp(num_users, num_items,  layers = [20,10], reg_layers=[0,0]):
    assert len(layers) == len(reg_layers)
    num_layer = len(layers) #Number of layers in the MLP
    
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    
    # Embedding layers, embedding size is half of the first MLP layer
    user_embedding_layer = Embedding(input_dim = num_users, output_dim = int(layers[0]/2), name = 'user_embedding',
                                  embeddings_initializer='uniform', input_length=1)
    user_embedding = user_embedding_layer(user_input)
    
    item_embedding_layer = Embedding(input_dim = num_items, output_dim = int(layers[0]/2), name = 'item_embedding',
                                  embeddings_initializer='uniform', input_length=1)
    item_embedding = item_embedding_layer(item_input)
    
    # Flatten embeddings
    user_latent = Flatten()(user_embedding)
    item_latent = Flatten()(item_embedding)
    
    # Concatenate two embeddings
    vector = Concatenate()([user_latent, item_latent])
    
    # MLP layer
    for idx in range(1, num_layer):
        layer = Dense(layers[idx], kernel_regularizer= l2(reg_layers[idx]), activation='relu', name = 'layer%d' %idx)
        vector = layer(vector)
    
    # Final prediction layer
    output = Dense(1, activation='sigmoid', name = 'prediction')(vector)
    
    model = Model([user_input, item_input], output)
    
    return model

In [11]:
# NMF
def get_model_nmf(num_users, num_items,  mf_dim=10, mlp_layers=[20, 10], reg_mlp_layers=[1, 1], reg_mf=0):
    assert len(mlp_layers) == len(reg_mlp_layers)
    num_layer = len(mlp_layers) #Number of layers in the MLP
    
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    
    # Embedding layers, embedding size is half of the first MLP layer
    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = mf_dim, name = 'MF_user_embedding',
                                  embeddings_initializer='uniform', input_length=1)(user_input)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = mf_dim, name = 'MF_item_embedding',
                                  embeddings_initializer='uniform', input_length=1)(item_input)
    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = int(mlp_layers[0]/2), name = 'MLP_user_embedding',
                                  embeddings_initializer='uniform', input_length=1)(user_input)
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = int(mlp_layers[0]/2), name = 'MLP_item_embedding',
                                  embeddings_initializer='uniform', input_length=1)(item_input)
    
    # MF part
    MF_user_latent = Flatten()(MF_Embedding_User)
    MF_item_latent = Flatten()(MF_Embedding_Item)
    mf_vector = Multiply(name="MF_output")([MF_user_latent, MF_item_latent])
    
    
    # MLP part
    MLP_user_latent = Flatten()(MLP_Embedding_User)
    MLP_item_latent = Flatten()(MLP_Embedding_Item)
    mlp_vector = Concatenate()([MLP_user_latent, MLP_item_latent])
    for idx in range(1, num_layer):
        layer = Dense(mlp_layers[idx], kernel_regularizer= l2(reg_mlp_layers[idx]), activation='relu', name = 'MLP_layer%d_output' %idx)
        mlp_vector = layer(mlp_vector)
    
    # Concatenate MF and MLP
    predict_vector = Concatenate(axis = -1, name="MLP_MF_concatenate_layer")([mf_vector, mlp_vector])
    
    # Final prediction layer
    output = Dense(1, activation='sigmoid', name = 'prediction')(predict_vector)
    
    model = Model([user_input, item_input], output)
    
    return model

# Compile and fit the model

In [12]:
# model = get_model_dot_product(num_users, num_items, 20, [1,1])
# model = get_model_gmf(num_users, num_items, 20, [1,1])
# model = get_model_mlp(num_users, num_items, [40, 100, 100], [1, 1, 1])
model = get_model_nmf(num_users, num_items, mf_dim=10, mlp_layers=[40, 10], reg_mlp_layers=[1, 1], reg_mf=0)
    
#Optimizer
model.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy', metrics=[MeanSquaredError(), RootMeanSquaredError()])

In [13]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
MLP_user_embedding (Embedding)  (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
MLP_item_embedding (Embedding)  (None, 1, 20)        33640       item_input[0][0]                 
______________________________________________________________________________________________

In [14]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [15]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=3)

In [16]:
# Training
hist = model.fit(x = [np.array(user_input_training), np.array(item_input_training)], #input
                 y = np.array(labels_training), # labels 
                 validation_data = ([np.array(user_input_test), np.array(item_input_test)], np.array(labels_test)),
                 callbacks = [tensorboard_callback, early_stopping_callback],
                 batch_size=25, epochs=10, verbose=2, shuffle=True)

Epoch 1/10
18114/18114 - 37s - loss: 0.4303 - mean_squared_error: 0.1315 - root_mean_squared_error: 0.3626 - val_loss: 0.3835 - val_mean_squared_error: 0.1202 - val_root_mean_squared_error: 0.3467
Epoch 2/10
18114/18114 - 34s - loss: 0.3843 - mean_squared_error: 0.1236 - root_mean_squared_error: 0.3516 - val_loss: 0.3440 - val_mean_squared_error: 0.1099 - val_root_mean_squared_error: 0.3315
Epoch 3/10
18114/18114 - 31s - loss: 0.3756 - mean_squared_error: 0.1221 - root_mean_squared_error: 0.3494 - val_loss: 0.3523 - val_mean_squared_error: 0.1119 - val_root_mean_squared_error: 0.3345
Epoch 4/10
18114/18114 - 31s - loss: 0.3751 - mean_squared_error: 0.1219 - root_mean_squared_error: 0.3492 - val_loss: 0.3435 - val_mean_squared_error: 0.1096 - val_root_mean_squared_error: 0.3311
Epoch 5/10
18114/18114 - 32s - loss: 0.3753 - mean_squared_error: 0.1218 - root_mean_squared_error: 0.3490 - val_loss: 0.3408 - val_mean_squared_error: 0.1090 - val_root_mean_squared_error: 0.3302
Epoch 6/10
1811

In [17]:
# Run tensorboard --logdir logs/fit in terminal

In [18]:
model.evaluate(x = [np.array(user_input_training), np.array(item_input_training)], y = np.array(labels_training))

14152/14152 [==============================] - 19s 1ms/step - loss: 0.3393 - mean_squared_error: 0.1101 - root_mean_squared_error: 0.3318 0s - loss: 0.3402 - mean_squared_error: 


[0.33927974104881287, 0.11012396216392517, 0.3318493068218231]

In [19]:
model.evaluate(x = [np.array(user_input_test), np.array(item_input_test)], y = np.array(labels_test))

1474/1474 [==============================] - 2s 1ms/step - loss: 0.3381 - mean_squared_error: 0.1037 - root_mean_squared_error: 0.3221


[0.3381277620792389, 0.10372933745384216, 0.3220703899860382]

In [20]:
# Take the corresponding layer name
user_layer_name = 'MLP_user_embedding'
user_embedding_layer_model = tf.keras.Model(inputs=model.input,
                                       outputs=model.get_layer(user_layer_name).output)
item_layer_name = 'MLP_item_embedding'
item_embedding_layer_model = tf.keras.Model(inputs=model.input,
                                       outputs=model.get_layer(item_layer_name).output)

In [21]:
user_embedding = user_embedding_layer_model([X_train["user_id"].unique(), np.zeros(X_train["user_id"].nunique())])
item_embedding = item_embedding_layer_model([np.zeros(X_train["item_id"].nunique()), X_train['item_id'].unique()])

In [22]:
user_embedding

<tf.Tensor: shape=(943, 20), dtype=float32, numpy=
array([[ 0.208342  , -0.24624884, -0.00896637, ...,  0.4941237 ,
         1.247704  ,  1.2151544 ],
       [-0.8695981 ,  0.7802609 ,  0.39058524, ...,  0.6570227 ,
        -0.18461995, -0.6947603 ],
       [ 0.12732166,  1.7560297 , -1.7483283 , ...,  0.9138868 ,
         0.48202744, -1.0967463 ],
       ...,
       [ 2.217535  , -0.31542385,  0.08181551, ..., -1.394803  ,
         1.5558014 ,  0.5322381 ],
       [-1.3472519 , -0.14246857, -1.6205634 , ..., -1.4780706 ,
        -3.0016947 ,  0.443716  ],
       [ 0.8591271 , -1.1994821 ,  0.5723794 , ..., -2.580457  ,
        -0.42483413,  2.1435823 ]], dtype=float32)>

In [23]:
item_embedding

<tf.Tensor: shape=(1675, 20), dtype=float32, numpy=
array([[-3.3090112e+00, -1.1315795e+00,  2.9134538e+00, ...,
        -2.8204441e+00, -2.1480088e+00,  3.1088510e+00],
       [-7.3075652e-02,  8.1069148e-01,  2.1057263e-01, ...,
        -1.8124223e-01, -2.1475883e-01,  4.2721593e-01],
       [-8.4408857e-03,  1.4443563e-01,  7.5141559e-03, ...,
        -3.1886537e-02, -2.6483300e-01,  2.9887655e-01],
       ...,
       [ 9.8523188e+00, -7.3046303e+00, -1.1276729e+01, ...,
         1.0209695e+01,  1.1230894e+01, -1.0268100e+01],
       [ 1.0492901e+01, -8.4913979e+00, -1.1257580e+01, ...,
         1.1997350e+01,  1.1684444e+01, -1.0920278e+01],
       [ 9.2690878e+00, -6.3645940e+00, -1.0006619e+01, ...,
         1.0387606e+01,  9.9319592e+00, -1.0924037e+01]], dtype=float32)>